**More QC & Taxonomic Profiling**
( rearranged steps a bit for now) 
- running all PSTR together 

- this follows part of 1Assembly steps and 4Taxonomy
    - host coral, human, and symbiont removal 
    - kraken and bracken read recruitment (taxonomic abundances)
    - will NOT be assembling for now (skipping megahit step)

**See notes on QC in CBC_metagenomics/0QC_trimming-112023 nb**

### Running host, symbiont removal and kraken/bracken

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu-long  # Partition
#SBATCH -t 56:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/slurm-removaltax-pstr-%j.out  # %j = job ID

cd /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr || exit
module load miniconda/22.11.1-1
conda activate anvio-8

# 1)remove host from sample reads

#set general parameters:
SAMPLENAME="pstr"
SAMPLELIST="filtered_sampleids"
RAWREADSPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/qc_112023'
READSPATH="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}/assembly/host_removed"
mkdir -p $READSPATH
LISTPATH="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}"
#set step parameters 
GENOME="Ofav"
INPUTPATH="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/${GENOME}_genome"
INDEX="${GENOME}_DB"

#build a bowtie2 index from a known genome
# **using ofav since pstr is more closely related to ofav than mcav - (was only built for histat so need to redo for bowtie)
#bowtie2-build $INPUTPATH/GCF_002042975.1_ofav_dov_v1_genomic.fna $INPUTPATH/"$INDEX"

#loop through samples
while IFS= read -r SAMPLEID; do
#re-align reads back to the index
bowtie2 -p 8 -x $INPUTPATH/$INDEX -1 "$RAWREADSPATH"/"${SAMPLEID}_R1_001_val_1.fq" -2 "$RAWREADSPATH"/"${SAMPLEID}_R2_001_val_2.fq" -S $READSPATH/"${SAMPLEID}"_mapped_and_unmapped.sam
#convert sam file from bowtie to a bam file for processing
samtools view -bS $READSPATH/"${SAMPLEID}"_mapped_and_unmapped.sam > $READSPATH/"${SAMPLEID}"_mapped_and_unmapped.bam
#extract only the reads of which both do not match against the host genome
samtools view -b -f 12 -F 256 $READSPATH/"${SAMPLEID}"_mapped_and_unmapped.bam > $READSPATH/"${SAMPLEID}"_bothReadsUnmapped.bam
# sorts the file so both mates are together and then extracts them back as .fastq files
samtools sort -n -m 5G -@ 2 $READSPATH/"${SAMPLEID}"_bothReadsUnmapped.bam -o $READSPATH/"${SAMPLEID}"_bothReadsUnmapped_sorted.bam
samtools fastq -@ 8 $READSPATH/"${SAMPLEID}"_bothReadsUnmapped_sorted.bam \
    -1 $READSPATH/"${SAMPLEID}"_host_removed_R1.fastq \
    -2 $READSPATH/"${SAMPLEID}"_host_removed_R2.fastq \
    -0 /dev/null -s /dev/null -n
 if [ $? -eq 0 ]; then
        echo "host removal completed successfully for sample: $SAMPLEID"
    else
        echo "host removal encountered an error for sample: $SAMPLEID"
        exit 1  
    fi
done < "$LISTPATH/${SAMPLELIST}"
conda deactivate
echo "Host removal: All samples processed successfully."

# 2)remove symbiont and human seqs using fastq screen 
module load bowtie/2.4.5
conda activate fastq_screen
FASTQSCREEN='/home/brooke_sienkiewicz_student_uml_edu/.conda/envs/fastq_screen/share/fastq-screen-0.15.3-0'
OUTPUTDIR="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}/assembly/final_reads_filtered"

mkdir -p "$OUTPUTDIR"
if [ $? -ne 0 ]; then
    echo "Error: Failed to create output directory $OUTPUTDIR"
    exit 1
fi

while IFS= read -r SAMPLEID; do
$FASTQSCREEN/fastq_screen --nohits --aligner bowtie2 --conf $FASTQSCREEN/fastq_screen.conf --outdir $OUTPUTDIR \
$READSPATH/"${SAMPLEID}"_host_removed_R1.fastq $READSPATH/"${SAMPLEID}"_host_removed_R2.fastq;
 if [ $? -eq 0 ]; then
        echo "fastq_screen completed successfully for sample: $SAMPLEID"
    else
        echo "fastq_screen encountered an error for sample: $SAMPLEID"
        exit 1
    fi
# --nohits = output reads do not map to any genomes
done < "$LISTPATH/${SAMPLELIST}"
conda deactivate
echo "Symbiont, host removal: All samples processed successfully."



# 3)concatenate all f and r seqs into single file (1 for f, 1 for r)
conda activate assembly

FILTERREADS="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}/assembly/final_reads_filtered"


# Read the sample IDs from the file
while IFS= read -r SAMPLEID; do
    # Construct the file paths for forward and reverse reads
    FORWARD_READ="$FILTERREADS/${SAMPLEID}_host_removed_R1.tagged_filter.fastq"
    REVERSE_READ="$FILTERREADS/${SAMPLEID}_host_removed_R2.tagged_filter.fastq"
    # Check if the files exist before concatenating
    if [ -e "$FORWARD_READ" ]; then
        cat "$FORWARD_READ" >> "$LISTPATH/assembly/${SAMPLENAME}_reads_R1_ALL.fastq"
    else
        echo "Forward read file not found for sample $SAMPLEID"
    fi
    if [ -e "$REVERSE_READ" ]; then
        cat "$REVERSE_READ" >> "$LISTPATH/assembly/${SAMPLENAME}_reads_R2_ALL.fastq"
    else
        echo "Reverse read file not found for sample $SAMPLEID"
    fi
done < "$LISTPATH/${SAMPLELIST}"
conda deactivate 

# kraken and bracken 
# SKIPPING intermediate steps to go ahead and run kraken 
conda activate kraken
SAMPLENAME="pstr"

# set variables
DBNAME='/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard'
OUTDIR='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}/taxonomy/kraken_stddb'
THREADS=24
KMER_LEN=35 #(default)
READ_LEN=150

mkdir -p $OUTDIR
if [ $? -ne 0 ]; then
    echo "Error: Failed to create output directory $OUTPUTDIR"
    exit 1
fi

# classify each set of paired end reads against silva taxonomic database
while IFS= read -r SAMPLEID; do
kraken2 --db $DBNAME --threads $THREADS --report $OUTDIR/${SAMPLEID}.kreport2 --report-zero-counts --paired $FILTERREADS/"${SAMPLEID}"_host_removed_R1.tagged_filter.fastq $FILTERREADS/"${SAMPLEID}"_host_removed_R2.tagged_filter.fastq > $OUTDIR/${SAMPLEID}.kraken2
if [ $? -eq 0 ]; then
        echo "kraken2 completed successfully for sample: $SAMPLEID"
    else
        echo "kraken2 encountered an error for sample: $SAMPLEID"
        exit 1
    fi
done < "$LISTPATH/${SAMPLELIST}"
echo "Kraken2: All samples processed successfully."

#'report-zero-counts' so that all reports have same taxa so they can be merged

# run bracken
# navigate to Bracken dir because can't figure out how to set up symlink 
cd //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/Bracken
# Generate bracken database - just do once
#bracken-build -d ${DBNAME} -t ${THREADS} -k ${KMER_LEN} -l ${READ_LEN} 
# Already Done

# Abundance Estimation
FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/${SAMPLENAME}/taxonomy/kraken_stddb'
LEVEL="S"

while IFS= read -r line; do
    INPUT_FILE="$FILEPATH/$line.kreport2"
    OUTPUT_FILE="$FILEPATH/${line%.kreport2}_$LEVEL.bracken"
    
    if [ -f "$INPUT_FILE" ]; then
        echo "Processing sample: $line"
        bracken -d "$DBNAME" -i "$INPUT_FILE" -o "$OUTPUT_FILE" -r "$READ_LEN" -l "$LEVEL"
        
        if [ $? -eq 0 ]; then
            echo "bracken completed successfully for sample: $line"
        else
            echo "bracken encountered an error for sample: $line"
            exit 1
        fi
    else
        echo "Error: Input file $INPUT_FILE does not exist"
        exit 1
    fi
done < "$LISTPATH/$SAMPLELIST"

echo "Bracken: All samples processed successfully."
#${SAMPLE}.kreport - the kraken report generated for a given dataset
#{BRACKEN_OUTPUT_FILE}.bracken - the desired name of the output file to be generated by the code
#The following optional parameters may be specified:
    #${LEVEL} - Default = 'S'. This specifies that abundance estimation will calculate estimated reads for each species. Other possible options are K (kingdom level), P (phylum), C (class), O (order), F (family), and G (genus).
    #${THRESHOLD} - Default = 10. For species classification, any species with <= 10 (or otherwise specified) reads will not receive any additional reads from higher taxonomy levels when distributing reads for abundance estimation. 
    #If another classification level is specified, thresholding will occur at that level.


# JOB-ID: 23190073, 23191990
# 23350054 - final (hopefully) script (minus host-removed since that was completed with job 23191990)
# ^didn't use filtered reads as input for kraken
# 23397902: just redoing #3 and #4 w/ filtered seqs (script file 'kraken')
# bash script file name: /work/pi_sarah_gignouxwolfsohn/brooke/${SAMPLENAME}/removaltax

In [ ]:
# HAVE NOT RUN YET 

# 4)ASSEMBLE reads into contigs (contiguous sequence - joins them together based on read overlap, and ensures there are no gaps
megahit --presets meta-large \
-1 "$OUTDIR"/"$SAMPLENAME"_reads_R1_ALL.fastq \
-2 "$OUTDIR"/"$SAMPLENAME"_reads_R2_ALL.fastq \
--keep-tmp-files \
#--continue \
-o $OUTDIR/contigs --out-prefix $SAMPLENAME 
#this one has to make the directory; will fail if it already exists
if [ $? -eq 0 ]; then
        echo "megahit completed successfully for samples: $SAMPLENAME"
    else
        echo "megahit encountered an error for samples: $SAMPLENAME"
        exit 1
    fi
conda deactivate 
echo "Megahit: Contig file created successfully."
#final assembled contigs: ${SAMPLENAME}/assembly/contigs


In [ ]:
# this was done previously in 11/2023 with no host removal or anything..delete after redoing process

#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu-long  # Partition
#SBATCH -t 56:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate assembly

#set up variables
READSPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/11272023/trimmed
SAMPLENAME=CBC
OUTDIR="$SAMPLENAME"_assembly

#concatenate all forward reads to one fastq file
cat "$READSPATH"/*_R1*.fq > "$READSPATH"/"$SAMPLENAME"_reads_R1_ALL.fastq

#concatenate all reverse reads to one fastq file
cat "$READSPATH"/*_R2*.fq > "$READSPATH"/"$SAMPLENAME"_reads_R2_ALL.fastq

megahit --presets meta-large \
-1 "$READSPATH"/"$SAMPLENAME"_reads_R1_ALL.fastq \
-2 "$READSPATH"/"$SAMPLENAME"_reads_R2_ALL.fastq \
--keep-tmp-files \
-o /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/11272023/$OUTDIR --out-prefix $SAMPLENAME 
#makes dir or do you need to mkdir beforehand???
#file: /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/11_2023bash_scripts/assembly
#JOB ID: 15972157

### Kraken results 
- very low classification rates 

In [ ]:
Loading database information... done.
28989033 sequences (5475.14 Mbp) processed in 69.191s (25138.2 Kseq/m, 4747.84 Mbp/m).
  833336 sequences classified (2.87%)
  28155697 sequences unclassified (97.13%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_45_PSTR
Loading database information... done.
20360853 sequences (3611.43 Mbp) processed in 46.989s (25998.5 Kseq/m, 4611.38 Mbp/m).
  503123 sequences classified (2.47%)
  19857730 sequences unclassified (97.53%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_28_PSTR
Loading database information... done.
14495243 sequences (2695.94 Mbp) processed in 35.030s (24827.9 Kseq/m, 4617.70 Mbp/m).
  2425666 sequences classified (16.73%)
  12069577 sequences unclassified (83.27%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_29_PSTR
Loading database information... done.
4061599 sequences (789.19 Mbp) processed in 10.412s (23404.4 Kseq/m, 4547.60 Mbp/m).
  146335 sequences classified (3.60%)
  3915264 sequences unclassified (96.40%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_46_PSTR
Loading database information... done.
16725843 sequences (3058.18 Mbp) processed in 40.679s (24669.8 Kseq/m, 4510.67 Mbp/m).
  340778 sequences classified (2.04%)
  16385065 sequences unclassified (97.96%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T1_4_PSTR
Loading database information... done.
21577306 sequences (4067.04 Mbp) processed in 53.055s (24401.9 Kseq/m, 4599.43 Mbp/m).
  464294 sequences classified (2.15%)
  21113012 sequences unclassified (97.85%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_25_PSTR
Loading database information... done.
26340368 sequences (4843.28 Mbp) processed in 62.979s (25094.3 Kseq/m, 4614.15 Mbp/m).
  589534 sequences classified (2.24%)
  25750834 sequences unclassified (97.76%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T3_15_PSTR
Loading database information... done.
38649413 sequences (6797.43 Mbp) processed in 90.692s (25569.8 Kseq/m, 4497.07 Mbp/m).
  924266 sequences classified (2.39%)
  37725147 sequences unclassified (97.61%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T3_3_PSTR
Loading database information... done.
18883375 sequences (3625.39 Mbp) processed in 46.018s (24620.8 Kseq/m, 4726.91 Mbp/m).
  410368 sequences classified (2.17%)
  18473007 sequences unclassified (97.83%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_27_PSTR
Loading database information... done.
21038048 sequences (4130.87 Mbp) processed in 53.740s (23488.9 Kseq/m, 4612.10 Mbp/m).
  539903 sequences classified (2.57%)
  20498145 sequences unclassified (97.43%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_14_PSTR
Loading database information... done.
22885217 sequences (4066.17 Mbp) processed in 54.918s (25003.0 Kseq/m, 4442.44 Mbp/m).
  539185 sequences classified (2.36%)
  22346032 sequences unclassified (97.64%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T1_10_PSTR
Loading database information... done.
9650681 sequences (1662.86 Mbp) processed in 22.238s (26038.3 Kseq/m, 4486.52 Mbp/m).
  180594 sequences classified (1.87%)
  9470087 sequences unclassified (98.13%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_26_PSTR
Loading database information... done.
18532636 sequences (3353.83 Mbp) processed in 44.284s (25109.7 Kseq/m, 4544.07 Mbp/m).
  682659 sequences classified (3.68%)
  17849977 sequences unclassified (96.32%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_13_PSTR
Loading database information... done.
21546121 sequences (3664.31 Mbp) processed in 50.901s (25397.8 Kseq/m, 4319.36 Mbp/m).
  476625 sequences classified (2.21%)
  21069496 sequences unclassified (97.79%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T1_11_PSTR
Loading database information... done.
12691364 sequences (2082.93 Mbp) processed in 28.553s (26668.9 Kseq/m, 4376.94 Mbp/m).
  229063 sequences classified (1.80%)
  12462301 sequences unclassified (98.20%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_32_PSTR
Loading database information... done.
27584708 sequences (5076.90 Mbp) processed in 65.510s (25264.4 Kseq/m, 4649.86 Mbp/m).
  597929 sequences classified (2.17%)
  26986779 sequences unclassified (97.83%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_30_PSTR
Loading database information... done.
29442462 sequences (5713.92 Mbp) processed in 74.174s (23816.4 Kseq/m, 4622.06 Mbp/m).
  653351 sequences classified (2.22%)
  28789111 sequences unclassified (97.78%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_33_PSTR
Loading database information... done.
16178429 sequences (2853.96 Mbp) processed in 38.795s (25021.4 Kseq/m, 4413.92 Mbp/m).
  348045 sequences classified (2.15%)
  15830384 sequences unclassified (97.85%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T1_31_PSTR
Loading database information... done.
18967626 sequences (3539.32 Mbp) processed in 45.938s (24773.6 Kseq/m, 4622.71 Mbp/m).
  405022 sequences classified (2.14%)
  18562604 sequences unclassified (97.86%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_30_PSTR
Loading database information... done.
9488210 sequences (1668.36 Mbp) processed in 22.954s (24801.9 Kseq/m, 4361.03 Mbp/m).
  186817 sequences classified (1.97%)
  9301393 sequences unclassified (98.03%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_31_PSTR
Loading database information... done.
22180483 sequences (4159.86 Mbp) processed in 54.826s (24273.5 Kseq/m, 4552.39 Mbp/m).
  465266 sequences classified (2.10%)
  21715217 sequences unclassified (97.90%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_37_PSTR
Loading database information... done.
27869329 sequences (4919.97 Mbp) processed in 65.213s (25641.3 Kseq/m, 4526.65 Mbp/m).
  568439 sequences classified (2.04%)
  27300890 sequences unclassified (97.96%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_36_PSTR
Loading database information... done.
9824136 sequences (1938.33 Mbp) processed in 24.947s (23627.6 Kseq/m, 4661.79 Mbp/m).
  583528 sequences classified (5.94%)
  9240608 sequences unclassified (94.06%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_7_PSTR
Loading database information... done.
30301892 sequences (5284.22 Mbp) processed in 70.163s (25912.8 Kseq/m, 4518.83 Mbp/m).
  595304 sequences classified (1.96%)
  29706588 sequences unclassified (98.04%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_36_PSTR
Loading database information... done.
26415194 sequences (5130.78 Mbp) processed in 67.417s (23509.1 Kseq/m, 4566.32 Mbp/m).
  564014 sequences classified (2.14%)
  25851180 sequences unclassified (97.86%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_40_PSTR
Loading database information... done.
22883705 sequences (4330.27 Mbp) processed in 56.080s (24483.2 Kseq/m, 4632.95 Mbp/m).
  483196 sequences classified (2.11%)
  22400509 sequences unclassified (97.89%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_35_PSTR
Loading database information... done.
23757549 sequences (4019.58 Mbp) processed in 55.783s (25553.7 Kseq/m, 4323.48 Mbp/m).
  519015 sequences classified (2.18%)
  23238534 sequences unclassified (97.82%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T2_34_PSTR
Loading database information... done.
26782526 sequences (4751.13 Mbp) processed in 62.857s (25565.1 Kseq/m, 4535.15 Mbp/m).
  548177 sequences classified (2.05%)
  26234349 sequences unclassified (97.95%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_34_PSTR
Loading database information... done.
24716567 sequences (4734.90 Mbp) processed in 59.948s (24738.2 Kseq/m, 4739.05 Mbp/m).
  584728 sequences classified (2.37%)
  24131839 sequences unclassified (97.63%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T3_23_PSTR
Loading database information... done.
32038239 sequences (5581.09 Mbp) processed in 74.541s (25788.5 Kseq/m, 4492.39 Mbp/m).
  593633 sequences classified (1.85%)
  31444606 sequences unclassified (98.15%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_38_PSTR
Loading database information... done.
31619956 sequences (5815.12 Mbp) processed in 78.197s (24261.8 Kseq/m, 4461.91 Mbp/m).
  732345 sequences classified (2.32%)
  30887611 sequences unclassified (97.68%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T2_8_PSTR
Loading database information... done.
15034256 sequences (2714.37 Mbp) processed in 36.043s (25026.9 Kseq/m, 4518.50 Mbp/m).
  323816 sequences classified (2.15%)
  14710440 sequences unclassified (97.85%)
kraken2 completed successfully for sample: 102019_BEL_CBC_T3_39_PSTR
Loading database information... done.
36404364 sequences (6443.79 Mbp) processed in 85.013s (25693.1 Kseq/m, 4547.84 Mbp/m).
  831601 sequences classified (2.28%)
  35572763 sequences unclassified (97.72%)
kraken2 completed successfully for sample: 052022_BEL_CBC_T1_55_PSTR
Kraken2: All samples processed successfully.

### Merge samples into single otu tables 
- goal: create taxa bar plots comparing composition and abundance between samples

In [1]:
import numpy as np
import pandas as pd
import os

##### all pstr

In [4]:
os.chdir('//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb')

In [2]:
# Set path variables and create sample list 

SAMPLES="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/filtered_sampleids"
BRACKENS='//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb'
samplelist=[]

with open(SAMPLES) as samples:
    for line in samples:
        samplelist.append(line.strip())
print(samplelist)

sample_names_string = ','.join(samplelist)
print(sample_names_string)
print(len(samplelist))

['052022_BEL_CBC_T2_45_PSTR', '102019_BEL_CBC_T1_28_PSTR', '102019_BEL_CBC_T1_29_PSTR', '052022_BEL_CBC_T2_46_PSTR', '052022_BEL_CBC_T1_4_PSTR', '102019_BEL_CBC_T1_25_PSTR', '052022_BEL_CBC_T3_15_PSTR', '052022_BEL_CBC_T3_3_PSTR', '102019_BEL_CBC_T1_27_PSTR', '052022_BEL_CBC_T2_14_PSTR', '052022_BEL_CBC_T1_10_PSTR', '102019_BEL_CBC_T1_26_PSTR', '052022_BEL_CBC_T2_13_PSTR', '052022_BEL_CBC_T1_11_PSTR', '102019_BEL_CBC_T2_32_PSTR', '102019_BEL_CBC_T1_30_PSTR', '102019_BEL_CBC_T2_33_PSTR', '102019_BEL_CBC_T1_31_PSTR', '102019_BEL_CBC_T2_30_PSTR', '102019_BEL_CBC_T2_31_PSTR', '102019_BEL_CBC_T3_37_PSTR', '102019_BEL_CBC_T2_36_PSTR', '052022_BEL_CBC_T2_7_PSTR', '102019_BEL_CBC_T3_36_PSTR', '102019_BEL_CBC_T3_40_PSTR', '102019_BEL_CBC_T3_35_PSTR', '102019_BEL_CBC_T2_34_PSTR', '102019_BEL_CBC_T3_34_PSTR', '052022_BEL_CBC_T3_23_PSTR', '102019_BEL_CBC_T3_38_PSTR', '052022_BEL_CBC_T2_8_PSTR', '102019_BEL_CBC_T3_39_PSTR', '052022_BEL_CBC_T1_55_PSTR']
052022_BEL_CBC_T2_45_PSTR,102019_BEL_CBC_T1_28

In [11]:
#create symlink
!ln -s //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/Bracken/bracken  //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/bracken
!ln -s //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/Bracken/bracken-build  //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/bracken-build
!cp //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/Bracken/analysis_scripts/combine_bracken_outputs.py   //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/combine_bracken_outputs.py

In [14]:
# combine individual kraken/bracken results to single otu table at species level

%run combine_bracken_outputs.py \
--files $BRACKENS/*.bracken \
--names $sample_names_string \
--output $BRACKENS/merged_bracken_species.txt

PROGRAM START TIME: 06-23-2024 18:48:52
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/052022_BEL_CBC_T2_13_PSTR_S.bracken:: Sample 052022_BEL_CBC_T2_45_PSTR
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/102019_BEL_CBC_T3_39_PSTR_S.bracken:: Sample 102019_BEL_CBC_T1_28_PSTR
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/052022_BEL_CBC_T2_14_PSTR_S.bracken:: Sample 102019_BEL_CBC_T1_29_PSTR
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/102019_BEL_CBC_T2_32_PSTR_S.bracken:: Sample 052022_BEL_CBC_T2_46_PSTR
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/102019_BEL_CBC_T3_37_PSTR_S.bracken:: Sample 052022_BEL_CBC_T1_4_PSTR
Processing Output File //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/taxonomy/kraken_stddb/052022

In [5]:
otu_table=pd.read_csv("merged_bracken_species.txt", sep='\t')
otu_table.set_index("taxonomy_id",inplace=True)
otu_table.head()

# Columns: 
# Sample num: Number of reads estimated for that species  
# Sample frac: Fraction of total reads in the sample estimated for this species
#       [NOTE: Fractions are of total reads classified, unclassified reads not accounted for]

,name,taxonomy_lvl,052022_BEL_CBC_T2_45_PSTR_num,052022_BEL_CBC_T2_45_PSTR_frac,102019_BEL_CBC_T1_28_PSTR_num,102019_BEL_CBC_T1_28_PSTR_frac,102019_BEL_CBC_T1_29_PSTR_num,102019_BEL_CBC_T1_29_PSTR_frac,052022_BEL_CBC_T2_46_PSTR_num,052022_BEL_CBC_T2_46_PSTR_frac,...,052022_BEL_CBC_T3_23_PSTR_num,052022_BEL_CBC_T3_23_PSTR_frac,102019_BEL_CBC_T3_38_PSTR_num,102019_BEL_CBC_T3_38_PSTR_frac,052022_BEL_CBC_T2_8_PSTR_num,052022_BEL_CBC_T2_8_PSTR_frac,102019_BEL_CBC_T3_39_PSTR_num,102019_BEL_CBC_T3_39_PSTR_frac,052022_BEL_CBC_T1_55_PSTR_num,052022_BEL_CBC_T1_55_PSTR_frac
taxonomy_id,,,,,,,,,,,,,,,,,,,,,
2775403,Rhizobium sp. TH2,S,57,0.00008,9,0.00003,18,0.00003,1,0.00000,...,3,0.00001,1,0.00001,12,0.00002,14,0.00002,4,0.00001
2952477,Rhizobium sp. SSA_523,S,53,0.00008,3,0.00001,20,0.00004,3,0.00001,...,9,0.00003,1,0.00001,11,0.00002,11,0.00001,16,0.00003
2978974,Rhizobium sp. CSC1952,S,55,0.00008,4,0.00001,22,0.00004,2,0.00001,...,5,0.00001,0,0.00000,9,0.00002,13,0.00001,11,0.00002
3039156,Rhizobium sp. CB3090,S,49,0.00007,3,0.00001,9,0.00002,1,0.00000,...,3,0.00001,11,0.00008,4,0.00001,3,0.00000,1,0.00000
2806346,Rhizobium sp. SL42,S,48,0.00007,0,0.00000,16,0.00003,1,0.00000,...,2,0.00001,0,0.00000,1,0.00000,5,0.00001,1,0.00000


In [6]:
# Make taxa otu id table 
otu_id=otu_table.iloc[:, 0:2]
otu_id
# export otu id 
otu_id.to_csv("otu_id_species")

In [7]:
# ID OTUnum for homo sapien
humanid=otu_id['name']=="Homo sapiens"
otu_id.loc[humanid,:]

,name,taxonomy_lvl
taxonomy_id,,
9606,Homo sapiens,S


In [20]:
# export raw reads file 
# drop name, taxa, and frac cols
drop_columns = [col for col in otu_table.columns if 'frac' in col]
drop_columns.extend(['name', 'taxonomy_lvl'])
otu_table_species=otu_table.drop(columns=drop_columns)

# rename to drop _num
otu_table_species.columns.str.replace('_num', '')

# remove homo sapien (9606)
otu_table_species.drop(9606,axis=0,inplace=True)
# export otu table to csv
otu_table_species.to_csv("otu_table_species")

In [19]:
# still need to normalize based on total number of reads???
# add column for normalized abundances for TOTAL number of reads per sample

import re
slurmdir='/work/pi_sarah_gignouxwolfsohn_uml_edu/brooke/pstr/slurm-kraken-pstr-23397902.out'

# Read the output report and extract the total number of reads
with open(slurmdir, 'r') as file:
    report_text = file.read()

# Extract total number of reads using regular expression
total_reads_pattern = re.compile(r'(\d+) sequences \(.* Mbp\) processed')
total_reads_matches = total_reads_pattern.findall(report_text)

# Create a dictionary to map sample ID to total reads
total_reads_dict = {samplelist[i]: int(total_reads_matches[i]) for i in range(len(samplelist))}


# Display the updated DataFrame
print(total_reads_dict)

# code adapted from chatgpt

{'052022_BEL_CBC_T2_45_PSTR': 28989033, '102019_BEL_CBC_T1_28_PSTR': 20360853, '102019_BEL_CBC_T1_29_PSTR': 14495243, '052022_BEL_CBC_T2_46_PSTR': 4061599, '052022_BEL_CBC_T1_4_PSTR': 16725843, '102019_BEL_CBC_T1_25_PSTR': 21577306, '052022_BEL_CBC_T3_15_PSTR': 26340368, '052022_BEL_CBC_T3_3_PSTR': 38649413, '102019_BEL_CBC_T1_27_PSTR': 18883375, '052022_BEL_CBC_T2_14_PSTR': 21038048, '052022_BEL_CBC_T1_10_PSTR': 22885217, '102019_BEL_CBC_T1_26_PSTR': 9650681, '052022_BEL_CBC_T2_13_PSTR': 18532636, '052022_BEL_CBC_T1_11_PSTR': 21546121, '102019_BEL_CBC_T2_32_PSTR': 12691364, '102019_BEL_CBC_T1_30_PSTR': 27584708, '102019_BEL_CBC_T2_33_PSTR': 29442462, '102019_BEL_CBC_T1_31_PSTR': 16178429, '102019_BEL_CBC_T2_30_PSTR': 18967626, '102019_BEL_CBC_T2_31_PSTR': 9488210, '102019_BEL_CBC_T3_37_PSTR': 22180483, '102019_BEL_CBC_T2_36_PSTR': 27869329, '052022_BEL_CBC_T2_7_PSTR': 9824136, '102019_BEL_CBC_T3_36_PSTR': 30301892, '102019_BEL_CBC_T3_40_PSTR': 26415194, '102019_BEL_CBC_T3_35_PSTR': 22

In [22]:
# Convert main df to raw abundances 
normalized_otus=otu_table.loc[:,otu_table.columns.str.contains("num")]
normalized_otus.columns = normalized_otus.columns.str.replace('_num', '')

# Convert the dictionary to a DataFrame and transpose it
sample_counts_df = pd.DataFrame(total_reads_dict, index=['Total_reads'])

# Concatenate the original DataFrame and the new DataFrame with the total reads row
normalized_otus = pd.concat([normalized_otus, sample_counts_df],axis=0)
normalized_otus

# Normalize: divide reads by total reads and multiply by 1 million
normalized_otu = (normalized_otus / normalized_otus.loc['Total_reads'] * 1E6)
# replace total reads row again 
normalized_otu.loc['Total_reads',:]=normalized_otus.loc['Total_reads',:]
normalized_otu["Sum"]=normalized_otu.sum(axis=1)
normalized_otu.sort_values(by="Sum", ascending=False)

,052022_BEL_CBC_T2_45_PSTR,102019_BEL_CBC_T1_28_PSTR,102019_BEL_CBC_T1_29_PSTR,052022_BEL_CBC_T2_46_PSTR,052022_BEL_CBC_T1_4_PSTR,102019_BEL_CBC_T1_25_PSTR,052022_BEL_CBC_T3_15_PSTR,052022_BEL_CBC_T3_3_PSTR,102019_BEL_CBC_T1_27_PSTR,052022_BEL_CBC_T2_14_PSTR,...,102019_BEL_CBC_T3_40_PSTR,102019_BEL_CBC_T3_35_PSTR,102019_BEL_CBC_T2_34_PSTR,102019_BEL_CBC_T3_34_PSTR,052022_BEL_CBC_T3_23_PSTR,102019_BEL_CBC_T3_38_PSTR,052022_BEL_CBC_T2_8_PSTR,102019_BEL_CBC_T3_39_PSTR,052022_BEL_CBC_T1_55_PSTR,Sum
Total_reads,2.898903e+07,2.036085e+07,1.449524e+07,4.061599e+06,1.672584e+07,2.157731e+07,2.634037e+07,3.864941e+07,1.888338e+07,2.103805e+07,...,2.641519e+07,2.288370e+07,2.375755e+07,2.678253e+07,2.471657e+07,3.203824e+07,3.161996e+07,1.503426e+07,3.640436e+07,7.279167e+08
9606,7.555857e+03,5.887916e+03,1.331044e+04,2.255688e+04,1.094193e+04,9.801641e+03,4.783380e+03,4.166143e+03,4.590864e+03,8.851582e+03,...,9.783990e+03,7.644304e+03,9.618585e+03,6.644482e+03,5.213426e+03,2.083666e+03,7.741250e+03,2.072826e+04,6.388273e+03,3.152261e+05
28901,7.211003e+02,2.143329e+02,1.781757e+03,1.621283e+03,1.891923e+03,3.995355e+03,3.615743e+02,1.275957e+03,1.024764e+03,9.711928e+02,...,3.772071e+02,8.007882e+02,1.435586e+03,1.474469e+02,8.316689e+02,7.800054e+01,6.569902e+02,1.497713e+03,8.479478e+02,4.716290e+04
1280,1.556796e+02,1.809895e+03,6.838795e+02,9.909890e+02,2.862038e+02,1.157698e+02,2.335958e+02,8.543467e+01,1.610411e+02,4.377783e+02,...,1.507769e+03,2.752614e+02,6.912750e+02,2.809332e+03,3.176412e+02,4.366345e+02,1.316257e+02,9.032838e+03,2.284891e+02,3.226823e+04
1872427,2.759664e+00,3.437970e-01,3.035479e+00,1.477251e+00,5.978772e-01,1.019590e+00,4.555745e-01,4.657251e-01,5.401577e+00,1.283389e+00,...,7.949970e-01,5.680898e-01,1.052297e+00,4.107156e-01,2.022935e-01,6.242540e-02,4.743840e-01,1.064236e+00,7.416693e-01,2.335709e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982266,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1982252,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
702738,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3050249,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [30]:
# remove taxa where sum is 0 and remove human??
# homo sapien is 9606
normalized_otu.drop(9606,axis=0,inplace=True)

In [31]:
# remove total reads row and sum col
normalized_otu.drop("Total_reads",axis=0,inplace=True)
normalized_otu.drop("Sum",axis=1,inplace=True)

In [32]:
# export otu table to csv
normalized_otu.to_csv("otu_normtable_species")